In [2]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split

from keras import Model , Sequential
from keras.layers import Dense

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go


c:\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def readData():
    #Hack to get path of Notebook (Data file is in same directory structure)
    
    if '_dh' in globals():
        current_folder = globals()['_dh'][0]
    else:
        script_path=__file__
        current_folder=dirname(script_path)

    # Calculating path to the test data
    data_location = os.path.join(current_folder,'..' ,'data' , 'data_gen.xlsx')
    data_location = os.path.normpath(data_location)

    wb = load_workbook(data_location, data_only=True)
    ws=wb['data']

    data={'x':[], 'y':[]}
    cur_row=2

    while ws['A'+str(cur_row)].value != None:
        data['x'].append(ws['A'+str(cur_row)].value)
        data['y'].append(ws['B'+str(cur_row)].value)
        cur_row += 1
    df = pd.DataFrame(data)
    train, test =  train_test_split(df, test_size=0.2)
    return train, test

In [4]:
def plotData(train, test, predictions):
    init_notebook_mode(connected=True)
    series1 = go.Scatter(
    x = train['x'],
    y = train['y'],
    name = 'Train',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgb(0,0,152)'
        )
    )

    series2 = go.Scatter(
        x = test['x'],
        y = test['y'],
        name = 'Test',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(0,152,0)'
        )
    )

    series3 = go.Scatter(
        x = test['x'],
        y = predictions.flatten(),
        name = 'Predictions',
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgb(256,256,0)'
        )
    )
    
    data = [series1,  series3]
    
    layout = dict(title = 'Training , Test and predictions',
                  yaxis = dict(zeroline = False),
                  xaxis = dict(zeroline = False)
                 )
    
    fig = dict(data=data, layout=layout)
    
    iplot(fig)

In [39]:
def defineModel():
    model = Sequential()
    model.add(Dense(2, input_dim=1, activation='sigmoid'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [40]:
train, test=readData()

train_x = np.array(train['x'], dtype=np.float32)
train_y = np.array(train['y'], dtype=np.float32)

test_x = np.array(test['x'], dtype=np.float32)
test_y = np.array(test['y'], dtype=np.float32)

In [41]:
model = defineModel()

In [42]:
history=model.fit(train_x,train_y, epochs=2500, verbose=0, batch_size=32,validation_data=(test_x, test_y))

In [45]:
history

In [43]:
predictions = model.predict(test_x)

In [44]:
plotData(train, test, predictions)